## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-28-13-00-00 +0000,wsj,Britain Pushed Ahead With Green Power. Its Gri...,https://www.wsj.com/business/energy-oil/britai...
1,2025-12-28-13-00-00 +0000,wsj,Private Equity Has More Housecleaning to Do in...,https://www.wsj.com/finance/private-equity-has...
2,2025-12-28-12-59-21 +0000,bbc,Three killed in Sweden as major storm sweeps a...,https://www.bbc.com/news/articles/cdj817m37rgo...
3,2025-12-28-12-41-00 +0000,wsj,Myanmar Junta Seeks Legitimacy in Widely Conde...,https://www.wsj.com/world/asia/myanmar-junta-s...
4,2025-12-28-12-37-26 +0000,bbc,Late shopper rush drives Boxing Day sales traffic,https://www.bbc.com/news/articles/c997e2vrzz7o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2359/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
92,trump,18
43,dies,9
134,new,9
36,brigitte,7
37,bardot,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
14,2025-12-28-11-26-43 +0000,nyt,Zelensky to Meet With Trump at Mar-a-Lago Abou...,https://www.nytimes.com/2025/12/28/world/europ...,45
91,2025-12-28-00-00-00 +0000,wsj,President Trump overrode some on the religious...,https://www.wsj.com/politics/policy/how-trump-...,42
53,2025-12-28-05-53-06 +0000,bbc,"Zelensky says Russia 'doesn't want peace', ahe...",https://www.bbc.com/news/articles/c1dzvpg1341o...,41
63,2025-12-28-03-35-45 +0000,nyt,Russia Pummels Kyiv Before Trump-Zelensky Meeting,https://www.nytimes.com/2025/12/27/world/europ...,40
139,2025-12-27-14-29-52 +0000,wapo,Russian attack pummels Kyiv as Zelensky prepar...,https://www.washingtonpost.com/world/2025/12/2...,39


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
14,45,2025-12-28-11-26-43 +0000,nyt,Zelensky to Meet With Trump at Mar-a-Lago Abou...,https://www.nytimes.com/2025/12/28/world/europ...
22,36,2025-12-28-10-45-00 +0000,wsj,"Legendary French Actress, Sex Symbol Brigitte ...",https://www.wsj.com/arts-culture/film/legendar...
107,32,2025-12-27-21-27-46 +0000,nypost,"Kim Jong Un hails bond of ‘blood, life and dea...",https://nypost.com/2025/12/27/world-news/kim-j...
45,30,2025-12-28-08-59-41 +0000,nypost,Next major storm system threatens Midwest with...,https://nypost.com/2025/12/28/us-news/major-st...
46,27,2025-12-28-08-50-52 +0000,nypost,Utah mom Elleshia Seymour allegedly abducted h...,https://nypost.com/2025/12/28/us-news/utah-mom...
49,26,2025-12-28-07-37-08 +0000,bbc,Polls close on first day of Myanmar's widely c...,https://www.bbc.com/news/articles/cy7vly6gm76o...
91,24,2025-12-28-00-00-00 +0000,wsj,President Trump overrode some on the religious...,https://www.wsj.com/politics/policy/how-trump-...
21,22,2025-12-28-11-00-00 +0000,cbc,"In 2025, melting sea ice allowed a Canadian ic...",https://www.cbc.ca/news/canada/north/canadian-...
110,19,2025-12-27-20-57-25 +0000,nypost,"Iran declares ‘total war’ against US, Israel a...",https://nypost.com/2025/12/27/world-news/irani...
55,19,2025-12-28-05-33-21 +0000,nypost,"Australian cruise ship runs aground, strikes c...",https://nypost.com/2025/12/28/world-news/austr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
